## Final Project Submission

Please fill out:
* Student name: Victor Kigen
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [6]:
# Loading movie gross data from Box Office Mojo
bom_movie_gross_data = pd.read_csv('zippedData/bom.movie_gross.csv', encoding='ISO-8859-1', low_memory=False)

# Loading Rotten Tomatoes movie info data with tab-separated values
info_data = pd.read_csv('zippedData/rt.movie_info.tsv', delimiter='\t', encoding='ISO-8859-1', low_memory=False)

# Loading Rotten Tomatoes review data, also tab-separated
review_data = pd.read_csv('zippedData/rt.reviews.tsv', delimiter='\t', encoding='ISO-8859-1', low_memory=False)

# Loading TMDb (The Movie Database) movie data
tmdb_movie_data = pd.read_csv('zippedData/tmdb.movies.csv', encoding='ISO-8859-1', low_memory=False)

# Loading movie budget data from The Numbers database
budget_data = pd.read_csv('zippedData/tn.movie_budgets.csv', encoding='ISO-8859-1', low_memory=False)


In [7]:
bom_movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [8]:
bom_movie_gross_data[bom_movie_gross_data[['foreign_gross']].isnull().any(axis=1)]
bom_movie_gross_data['foreign_gross'] = pd.to_numeric(bom_movie_gross_data['foreign_gross'].replace('[\$,]', '', regex=True))
foreign_gross_median = bom_movie_gross_data['foreign_gross'].median() 

bom_movie_gross_data['foreign_gross'].fillna(foreign_gross_median, inplace=True)

In [9]:
bom_movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   3387 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [10]:
bom_movie_gross_data[bom_movie_gross_data[['domestic_gross']].isnull().any(axis=1)]
bom_movie_gross_data['domestic_gross'] = pd.to_numeric(bom_movie_gross_data['domestic_gross'].replace('[\$,]', '', regex=True))
domestic_gross_median = bom_movie_gross_data['domestic_gross'].median() 

bom_movie_gross_data['domestic_gross'].fillna(domestic_gross_median, inplace=True)
bom_movie_gross_data.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010


In [11]:
mode_value = bom_movie_gross_data['studio'].mode()[0]
bom_movie_gross_data['studio'].fillna(mode_value, inplace=True)


In [12]:
bom_movie_gross_data['studio'] = bom_movie_gross_data['studio'].str.upper()

In [13]:
bom_movie_gross_data['year'].value_counts()

2015    450
2016    436
2012    400
2011    399
2014    395
2013    350
2010    328
2017    321
2018    308
Name: year, dtype: int64

In [14]:
columns_to_drop = ['synopsis', 'dvd_date', 'studio', 'currency', 'box_office']
info_data = info_data.drop(columns=columns_to_drop)

info_data.head()


,id,rating,genre,director,writer,theater_date,runtime
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971",104 minutes
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996",116 minutes
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994",128 minutes
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,200 minutes


In [15]:
info_data[['director', 'writer']] = info_data[['director', 'writer']].fillna('N/A')

info_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   rating        1557 non-null   object
 2   genre         1552 non-null   object
 3   director      1560 non-null   object
 4   writer        1560 non-null   object
 5   theater_date  1201 non-null   object
 6   runtime       1530 non-null   object
dtypes: int64(1), object(6)
memory usage: 85.4+ KB


In [16]:
# Convert the date strings to datetime objects

info_data['theater_date'] = pd.to_datetime(info_data['theater_date'], errors='coerce')


date_mean = info_data['theater_date'].mean()
info_data['theater_date'].fillna(date_mean, inplace=True)

# Format the datetime objects to the desired string format
info_data['theater_date'] = info_data['theater_date'].dt.strftime('%Y-%m-%d')


In [17]:
info_data['runtime'] = pd.to_numeric(info_data['runtime'].replace(' minutes', '', regex=True))
info_data.head()

runtime_mean = info_data['runtime'].mean()
info_data['runtime'].fillna(runtime_mean, inplace=True)

mode_value = info_data['rating'].mode()[0]
info_data['rating'].fillna(mode_value, inplace=True)
info_data.head()

mode_value = info_data['genre'].mode()[0]
info_data['genre'].fillna(mode_value, inplace=True)
info_data.head()


,id,rating,genre,director,writer,theater_date,runtime
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,1971-10-09,104.0
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,108.0
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,1996-09-13,116.0
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,128.0
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,1992-03-03,200.0


In [18]:
review_data.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [19]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [20]:
review_data['rating'].unique()



array(['3/5', nan, 'C', '2/5', 'B-', '2/4', 'B', '3/4', '4/5', '4/4',
       '6/10', '1/4', '8', '2.5/4', '4/10', '2.0/5', '3/10', '7/10', 'A-',
       '5/5', 'F', '3.5/4', 'D+', '1.5/4', '3.5/5', '8/10', 'B+', '9/10',
       '2.5/5', '7.5/10', '5.5/10', 'C-', '1.5/5', '1/5', '5/10', 'C+',
       '0/5', '6', '0.5/4', 'D', '3.1/5', '3/6', '4.5/5', '0/4', '2/10',
       'D-', '7', '1/10', '3', 'A+', 'A', '4.0/4', '9.5/10', '2.5',
       '2.1/2', '6.5/10', '3.7/5', '8.4/10', '9', '1', '7.2/10', '2.2/5',
       '0.5/10', '5', '0', '2', '4.5', '7.7', '5.0/5', '8.5/10', '3.0/5',
       '0.5/5', '1.5/10', '3.0/4', '2.3/10', '4.5/10', '4/6', '3.5',
       '8.6/10', '6/8', '2.0/4', '2.7', '4.2/10', '5.8', '4', '7.1/10',
       '5/4', 'N', '3.5/10', '5.8/10', 'R', '4.0/5', '0/10', '5.0/10',
       '5.9/10', '2.4/5', '1.9/5', '4.9', '7.4/10', '1.5', '2.3/4',
       '8.8/10', '4.0/10', '2.2', '3.8/10', '6.8/10', '7.3', '7.0/10',
       '3.2', '4.2', '8.4', '5.5/5', '6.3/10', '7.6/10', '8.1/10',
  

In [21]:
for index, value in review_data['rating'].iteritems():
    if isinstance(value, str): 
        if '/' in value: 
            if ' ' in value: 
                whole, fraction = value.split(' ')
                numerator, denominator = fraction.split('/')
                new_rating = float(whole) + (float(numerator) / float(denominator))
                review_data.at[index, 'rating'] = round(new_rating, 2)

            else:  # Regular fraction
                numerator, denominator = value.split('/')
                new_rating = float(numerator) / float(denominator)
                review_data.at[index, 'rating'] = round(new_rating, 2)



In [22]:
review_data['top_critic'].unique()

review_data['date'] = pd.to_datetime(review_data['date'], errors='coerce')
review_data['date'] = review_data['date'].dt.strftime('%Y-%m-%d')

In [23]:
columns_to_drop = ['critic', 'fresh', 'publisher']
review_data = review_data.drop(columns=columns_to_drop)


In [24]:
tmdb_movie_data.head()


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [25]:
tmdb_movie_data['popularity'].unique()

array([33.533, 28.734, 28.515, ...,  0.612,  0.609,  0.607])

In [26]:
budget_data['production_budget'] = pd.to_numeric(budget_data['production_budget'].replace('[\$,]', '', regex=True))
budget_data['domestic_gross'] = pd.to_numeric(budget_data['domestic_gross'].replace('[\$,]', '', regex=True))
budget_data['worldwide_gross'] = pd.to_numeric(budget_data['worldwide_gross'].replace('[\$,]', '', regex=True))


In [27]:
# Convert the date strings to datetime objects

budget_data['release_date'] = pd.to_datetime(budget_data['release_date'], errors='coerce')

# Format the datetime objects to the desired string format
budget_data['release_date'] = budget_data['release_date'].dt.strftime('%Y-%m-%d')

In [28]:
budget_data.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


### Sql Table creation

In [36]:
import zipfile
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('unzippedData/')

In [37]:
# creating Joined sql database
conn = sqlite3.connect('unzippedData/im.db')
cur = conn.cursor()

- Checking the contents of every table in your sqlite database

In [38]:
q = """ 
SELECT *
FROM directors
"""

df_directors = pd.read_sql(q, conn)
df_directors.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [56]:
q = """ 
SELECT *
FROM movie_akas
"""

df_akas = pd.read_sql(q, conn)
df_akas.tail()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0
331702,tt9880178,3,The Attention,XWW,en,imdbDisplay,None,0.0


In [70]:
df_akas['region'].unique()

array(['BG', 'JP', 'BR', 'FR', 'GR', 'IT', 'HR', 'IL', 'MX', 'EE', 'SE',
       'US', 'DE', 'XWW', 'LV', 'PT', 'RU', 'CZ', 'CA', 'CL', 'RO', 'LT',
       'FI', 'PE', 'TR', 'NO', 'SI', 'UA', 'VN', 'DK', 'RS', 'SK', None,
       'ES', 'HU', 'PL', 'AZ', 'GE', 'UY', 'AR', 'GB', 'IN', 'CO', 'AU',
       'IE', 'AT', 'CN', 'KR', 'BA', 'VE', 'CH', 'DO', 'XEU', 'NL', 'BO',
       'JM', 'IR', 'XSA', 'MK', 'ID', 'TW', 'PH', 'TH', 'HK', 'PR', 'BE',
       'CU', 'AL', 'AN', 'EG', 'SG', 'IS', 'QA', 'NZ', 'AM', 'TN', 'MY',
       'SD', 'ZW', 'PS', 'MN', 'NG', 'SV', 'MA', 'BZ', 'UZ', 'AE', 'KP',
       'GH', 'PA', 'LU', 'ZA', 'CD', 'NP', 'LB', 'IQ', 'MH', 'EC', 'JO',
       'SN', 'BY', 'ET', 'PK', 'KZ', 'GP', 'ME', 'LK', 'DZ', 'VI', 'HN',
       'KH', 'CY', 'BD', 'AF', 'GT', 'XNA', 'CSHH', 'TT', 'SUHH', 'BH',
       'CV', 'PG', 'GU', 'TZ', 'MU', 'CR', 'PY', 'XKV', 'UG', 'AW', 'XAS',
       'BT', 'KE', 'SL', 'MD', 'RW', 'SY', 'BS', 'HT', 'MG', 'LA', 'MO',
       'MM', 'MW', 'SZ', 'AO', 'CG', 'KG', 'LR'

In [66]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [69]:
df_akas.isna().mean() *100

movie_id              0.000000
ordering              0.000000
title                 0.000000
region               16.066481
language             87.423991
types                49.217523
attributes           95.500493
is_original_title     0.007537
dtype: float64

In [64]:
df_akas['types'].unique()

array([None, 'imdbDisplay', 'working', 'original', 'alternative', 'video',
       'dvd', 'tv', 'festival', 'dvd\x02imdbDisplay',
       'festival\x02working'], dtype=object)

In [44]:
q = """ 
SELECT *
FROM movie_ratings
"""

df_ratings = pd.read_sql(q, conn)
df_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [50]:
q = """ 
SELECT *
FROM principals
"""

df_principals = pd.read_sql(q, conn)
df_principals.head()

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [51]:
q = """ 
SELECT *
FROM known_for
"""

df_known_for = pd.read_sql(q, conn)
df_known_for.head()

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [52]:
q = """ 
SELECT *
FROM movie_basics
"""

df_movie_basics = pd.read_sql(q, conn)
df_movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [71]:
df_movie_basics.columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [53]:
q = """ 
SELECT *
FROM persons
"""

df_persons = pd.read_sql(q, conn)
df_persons.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [54]:
q = """ 
SELECT *
FROM writers
"""

df_writers= pd.read_sql(q, conn)
df_writers.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


In [72]:
# joining sql tables
q = """ 
SELECT mb.primary_title, mb.original_title, mb.start_year,mb.runtime_minutes, mb.genres, mr.movie_id,	mr.averagerating,	mr.numvotes,ma.region
FROM movie_basics AS mb
JOIN movie_ratings AS mr
USING(movie_id)
JOIN movie_akas AS ma
USING (movie_id)
JOIN 

"""
data_df = pd.read_sql(q,conn)

In [73]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261806 entries, 0 to 261805
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    261806 non-null  object 
 1   original_title   261806 non-null  object 
 2   start_year       261806 non-null  int64  
 3   runtime_minutes  250553 non-null  float64
 4   genres           260621 non-null  object 
 5   movie_id         261806 non-null  object 
 6   averagerating    261806 non-null  float64
 7   numvotes         261806 non-null  int64  
 8   region           218341 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 18.0+ MB


In [74]:
# creating backup
df = data_df


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261806 entries, 0 to 261805
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   primary_title    261806 non-null  object 
 1   original_title   261806 non-null  object 
 2   start_year       261806 non-null  int64  
 3   runtime_minutes  250553 non-null  float64
 4   genres           260621 non-null  object 
 5   movie_id         261806 non-null  object 
 6   averagerating    261806 non-null  float64
 7   numvotes         261806 non-null  int64  
 8   region           218341 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 18.0+ MB


In [75]:
df.isna().mean()*100

primary_title       0.000000
original_title      0.000000
start_year          0.000000
runtime_minutes     4.298221
genres              0.452625
movie_id            0.000000
averagerating       0.000000
numvotes            0.000000
region             16.601988
dtype: float64

In [77]:
# filling missing values
df['runtime_minutes'].fillna(df['runtime_minutes'].mean(), inplace=True)
df.isna().mean()*100


primary_title       0.000000
original_title      0.000000
start_year          0.000000
runtime_minutes     0.000000
genres              0.452625
movie_id            0.000000
averagerating       0.000000
numvotes            0.000000
region             16.601988
dtype: float64

In [84]:
df['genres'].fillna('unknown', inplace=True)
df['region'].fillna('unknown', inplace=True)

df.isna().mean()*100


primary_title      0.0
original_title     0.0
start_year         0.0
runtime_minutes    0.0
genres             0.0
movie_id           0.0
averagerating      0.0
numvotes           0.0
region             0.0
dtype: float64